# GloVe Word Embeddings Demo

This demo was part of a presentation for [this word embeddings workshop](https://www.eventbrite.com/e/practical-ai-for-female-engineers-product-managers-and-designers-tickets-34805104003) and a talk at [the Demystifying AI conference](https://www.eventbrite.com/e/demystifying-deep-learning-ai-tickets-34351888423).  It is not necessary to download the demo to be able to follow along and enjoy the workshop.

It is available on Github at https://github.com/fastai/word-embeddings-workshop

## Loading our data

### Imports

In [3]:
import pickle
import numpy as np
import re
import json

In [4]:
np.set_printoptions(precision=4, suppress=True)

The dataset is available at http://files.fast.ai/models/glove/6B.100d.tgz
To download and unzip the files from the command line, you can run:

    wget http://files.fast.ai/models/glove_50_glove_100.tgz 
    tar xvzf glove_50_glove_100.tgz

You will need to update the path below to be accurate for where you are storing the data.

In [9]:
vecs = np.load("/Users/sdasgupta/Projects/word-embeddings-workshop/glove_50_glove_100/glove_vectors_50d.npy")
vecs50 = np.load("/Users/sdasgupta/Projects/word-embeddings-workshop/glove_50_glove_100/glove_vectors_50d.npy")

In [13]:
with open('/Users/sdasgupta/Projects/word-embeddings-workshop/glove_50_glove_100/words.txt') as f:
    content = f.readlines()
words = [x.strip() for x in content] 

In [15]:
wordidx = json.load(open('/Users/sdasgupta/Projects/word-embeddings-workshop/glove_50_glove_100/wordsidx.txt'))

### What the data looks like

Let's see what our data looks like:

In [16]:
len(words)

400000

In [17]:
words[:10]

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]

In [18]:
words[600:610]

['together',
 'congress',
 'index',
 'australia',
 'results',
 'hard',
 'hours',
 'land',
 'action',
 'higher']

wordidx allows us to look up a word in order to find out it's index:

In [19]:
type(wordidx)

dict

In [22]:
wordidx['queen']

2060

In [23]:
words[2060]

'queen'

## Words as vectors

The word "intelligence" is represented by the 100 dimensional vector:

In [24]:
type(vecs)

numpy.ndarray

In [25]:
vecs[11853]

array([-0.8073,  0.2036, -0.9182, -0.8927, -0.0399,  1.1941, -0.229 ,
       -1.2243, -0.2975,  0.9244, -0.1352, -0.716 , -0.1969,  0.9293,
       -0.291 , -0.806 ,  0.2371, -0.6206,  0.8621,  0.5686, -0.1533,
        1.3701, -0.3057,  0.9811,  0.8648, -0.9397, -1.3222, -0.9821,
       -1.0882,  0.3535,  1.0032, -0.8458, -0.354 , -1.0381, -1.6699,
       -0.9312, -1.0793, -0.1909, -0.1622, -0.133 ,  0.3638, -0.173 ,
        0.058 ,  0.528 ,  0.7011, -0.2533,  0.2567, -0.4472,  0.074 ,
        0.0105], dtype=float32)

This lets us do some useful calculations. For instance, we can see how far apart two words are using a distance metric:

In [26]:
from scipy.spatial.distance import cosine as dist

Smaller numbers mean two words are closer together, larger numbers mean they are further apart.

The distance between similar words is low:

In [27]:
dist(vecs[wordidx["puppy"]], vecs[wordidx["dog"]])

0.2245078682899475

In [28]:
dist(vecs[wordidx["queen"]], vecs[wordidx["princess"]])

0.14848339557647705

And the distance between unrelated words is high:

In [29]:
dist(vecs[wordidx["celebrity"]], vecs[wordidx["dusty"]])

0.8453244715929031

In [30]:
dist(vecs[wordidx["kitten"]], vecs[wordidx["airplane"]])

0.775413453578949

In [31]:
dist(vecs[wordidx["avalanche"]], vecs[wordidx["antique"]])

1.0188372414559126

### Bias

There is a lot of opportunity for bias:

In [20]:
dist(vecs[wordidx["man"]], vecs[wordidx["genius"]])

0.50985148631697985

In [21]:
dist(vecs[wordidx["woman"]], vecs[wordidx["genius"]])

0.6897833082810727

Not all pairs are stereotyped:

In [22]:
dist(vecs[wordidx["man"]], vecs[wordidx["emotional"]])

0.55957489609574407

In [23]:
dist(vecs[wordidx["woman"]], vecs[wordidx["emotional"]])

0.62572056015698596

I just checked the distance between pairs of words, because this is a quick and simple way to illustrate the concept.  It is also a very **noisy** approach, and **researchers approach this problem in more systematic ways**.

## Visualizing the words

We will use [Plotly](https://plot.ly/), a Python library to make interactive graphs (note: everything below is done with the free, offline version of Plotly).

### Methods

In [33]:
import plotly
import plotly.graph_objs as go    
from IPython.display import IFrame

In [34]:
def plotly_3d(Y, cat_labels):
    trace_dict = {}
    for i, label in enumerate(cat_labels):
        trace_dict[i] = go.Scatter3d(
            x=Y[i*5:(i+1)*5, 0],
            y=Y[i*5:(i+1)*5, 1],
            z=Y[i*5:(i+1)*5, 2],
            mode='markers',
            marker=dict(
                size=8,
                line=dict(
                    color='rgba('+ str(i*40) + ',' + str(i*40) + ',' + str(i*40) + ', 0.14)',
                    width=0.5
                ),
                opacity=0.8
            ),
            text = my_words[i*5:(i+1)*5],
            name = label
        )

    data = [item for item in trace_dict.values()]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    plotly.offline.plot({
        "data": data,
        "layout": layout
    })

In [35]:
def plotly_2d(Y, cat_labels):
    trace_dict = {}
    for i, label in enumerate(cat_labels):
        trace_dict[i] = go.Scatter(
            x=Y[i*5:(i+1)*5, 0],
            y=Y[i*5:(i+1)*5, 1],
            mode='markers',
            marker=dict(
                size=8,
                line=dict(
                    color='rgba('+ str(i*40) + ',' + str(i*40) + ',' + str(i*40) + ', 0.14)',
                    width=0.5
                ),
                opacity=0.8
            ),
            text = my_words[i*5:(i+1)*5],
            name = label
        )

    data = [item for item in trace_dict.values()]
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    plotly.offline.plot({
        "data": data,
        "layout": layout
    })

### Preparing the Data

Let's plot words from a few different categories:

In [36]:
categories = [
              "bugs", "music", 
              "pleasant", "unpleasant", 
              "science", "arts"
             ]

In [37]:
my_words = [
            "maggot", "flea", "tarantula", "bedbug", "mosquito", 
            "violin", "cello", "flute", "harp", "mandolin",
            "joy", "love", "peace", "pleasure", "wonderful",
            "agony", "terrible", "horrible", "nasty", "failure", 
            "physics", "chemistry", "science", "technology", "engineering",
            "poetry", "art", "literature", "dance", "symphony",
           ]

Again, we need to look up the indices of our words using the wordidx dictionary:

In [38]:
X = np.array([wordidx[word] for word in my_words])

In [39]:
vecs[X].shape

(30, 50)

Now, we will make a set combining our words with the first 10,000 words in our entire set of words (some of the words will already be in there), and create a matrix of their embeddings.

In [44]:
embeddings = np.concatenate((vecs[X], vecs[:100,:]), axis=0); embeddings.shape

(130, 50)

### Viewing the words in 3D

The words are in 100 dimensions, so we will need a way to reduce them to 3 dimensions so that we can view them.  Two good options are T-SNE or PCA.  The main idea is to find a meaningful way to go from 100 dimensions to 3 dimensions (while keeping a similar notion of what is close to what).

You would typically just use one of these (T-SNE or PCA).  I've included both if you're interested.

#### TSNE

In [48]:
from sklearn import manifold

In [49]:
tsne = manifold.TSNE(n_components=3, init='pca', random_state=0)
Y = tsne.fit_transform(embeddings)
plotly_3d(Y, categories)

In [ ]:
IFrame('temp-plot.html', width=600, height=400)

#### PCA

In [50]:
from sklearn import decomposition

In [51]:
pca = decomposition.PCA(n_components=3).fit(embeddings.T)
components = pca.components_
plotly_3d(components.T[:len(my_words),:], categories)

In [ ]:
IFrame('temp-plot.html', width=600, height=400)

## Nearest Neighbors

We can also see what words are close to a given word.

In [52]:
from sklearn.neighbors import NearestNeighbors

Nearest Neighbors is an algorithm that finds the points closest to a given point.

In [53]:
neigh = NearestNeighbors(n_neighbors=10, radius=0.5, metric='cosine', algorithm='brute')
neigh.fit(vecs) 

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=0.5)

In [155]:
distances, indices = neigh.kneighbors([vecs[wordidx["feminist"]]])

In [156]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('feminist', 0.0),
 ('feminism', 0.13520288),
 ('feminists', 0.22253186),
 ('activism', 0.22524899),
 ('humanist', 0.3072952),
 ('activist', 0.30948985),
 ('lesbian', 0.3148585),
 ('environmentalism', 0.3337556),
 ('left-wing', 0.33652592),
 ('libertarian', 0.33681267)]

We can take this a step further, and add two words together.  What is the result?

In [56]:
new_vec = vecs[wordidx["artificial"]] + vecs[wordidx["intelligence"]]

In [57]:
new_vec

array([ 1.5938,  0.252 ,  0.3125,  0.5391, -0.0901,  0.4677,  0.0858,
       -1.1492,  1.3933, -0.2219,  0.7984,  0.8358,  0.0552,  0.8999,
       -0.9726,  0.9242,  0.0584,  0.2309, -0.1326,  0.3531, -0.1392,
        0.8172,  1.0825, -1.6307,  0.083 , -2.6478, -0.8942,  0.1949,
        0.463 ,  0.3456,  4.7209, -0.9311, -0.7784, -2.4081, -0.1621,
        1.4523, -0.3381,  1.0582, -0.3803,  0.105 ,  0.5123,  0.05  ,
        0.4723,  1.1626,  0.6585, -0.4311,  0.3381,  1.7683, -0.3458,
       -0.3366], dtype=float32)

In [58]:
distances, indices = neigh.kneighbors([new_vec])

In [59]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('intelligence', 0.17358804),
 ('scientific', 0.2144543),
 ('biological', 0.21460235),
 ('knowledge', 0.22620225),
 ('source', 0.22885275),
 ('critical', 0.23267126),
 ('monitoring', 0.23394763),
 ('using', 0.23644698),
 ('develop', 0.2411797),
 ('information', 0.24139535)]

In [60]:
distances, indices = neigh.kneighbors([vecs[wordidx["king"]]])

In [61]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('king', 0.0),
 ('prince', 0.176382),
 ('queen', 0.21609563),
 ('ii', 0.22537696),
 ('emperor', 0.22637528),
 ('son', 0.2332806),
 ('uncle', 0.2372849),
 ('kingdom', 0.24578387),
 ('throne', 0.24600852),
 ('brother', 0.25075883)]

In [62]:
new_vec = vecs[wordidx["king"]] - vecs[wordidx["he"]] + vecs[wordidx["she"]]

In [63]:
distances, indices = neigh.kneighbors([new_vec])

In [64]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('king', 0.11406088),
 ('queen', 0.11687255),
 ('princess', 0.1953603),
 ('daughter', 0.22080821),
 ('elizabeth', 0.22723442),
 ('prince', 0.23495907),
 ('mother', 0.25153065),
 ('uncle', 0.2596851),
 ('aunt', 0.26034725),
 ('son', 0.26454055)]

In [65]:
wordidx["programmer"]

19226

In [66]:
distances, indices = neigh.kneighbors([vecs[wordidx["programmer"]]])

Closest words to "programmer":

In [67]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('programmer', 5.9604645e-08),
 ('programmers', 0.29829514),
 ('software', 0.3133769),
 ('computer', 0.33164066),
 ('prodigy', 0.34746027),
 ('animator', 0.351624),
 ('entrepreneur', 0.36156118),
 ('technician', 0.36351287),
 ('inventor', 0.3709517),
 ('hacker', 0.38514376)]

Feminine version of "programmer"

In [68]:
new_vec = vecs[wordidx["programmer"]] - vecs[wordidx["he"]] + vecs[wordidx["she"]]

In [69]:
distances, indices = neigh.kneighbors([new_vec])

In [70]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('programmer', 0.16746092),
 ('prodigy', 0.29029918),
 ('introduces', 0.3470044),
 ('therapist', 0.3516662),
 ('lets', 0.38770908),
 ('stylist', 0.4003443),
 ('narrator', 0.40137964),
 ('publicist', 0.4026603),
 ('acrobat', 0.40394717),
 ('learns', 0.4041685)]

Masculine version of "programmer"

In [71]:
new_vec = vecs[wordidx["programmer"]] - vecs[wordidx["she"]] + vecs[wordidx["he"]]

In [72]:
distances, indices = neigh.kneighbors([new_vec])

In [73]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('programmer', 0.15668714),
 ('programmers', 0.363079),
 ('software', 0.40670604),
 ('compiler', 0.4082281),
 ('setup', 0.4159652),
 ('innovator', 0.41614568),
 ('backup', 0.41730547),
 ('command-line', 0.41792),
 ('computer', 0.43415558),
 ('computing', 0.43467033)]

In [74]:
distances, indices = neigh.kneighbors([vecs[wordidx["doctor"]]])

In [75]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('doctor', 0.0),
 ('nurse', 0.20225024),
 ('physician', 0.20347887),
 ('patient', 0.2388308),
 ('child', 0.2441063),
 ('teacher', 0.24617153),
 ('surgeon', 0.25209272),
 ('psychiatrist', 0.25779068),
 ('doctors', 0.2606392),
 ('father', 0.26658934)]

Feminine version of doctor:

In [76]:
new_vec = vecs[wordidx["doctor"]] - vecs[wordidx["he"]] + vecs[wordidx["she"]]

In [77]:
distances, indices = neigh.kneighbors([new_vec])

In [78]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('doctor', 0.107596636),
 ('nurse', 0.15962988),
 ('mother', 0.19535291),
 ('child', 0.20671248),
 ('woman', 0.21084964),
 ('pregnant', 0.22029269),
 ('girl', 0.2237624),
 ('her', 0.24301541),
 ('she', 0.25414348),
 ('patient', 0.2599827)]

Masculine version of doctor:

In [79]:
new_vec = vecs[wordidx["doctor"]] - vecs[wordidx["she"]] + vecs[wordidx["he"]]

In [80]:
distances, indices = neigh.kneighbors([new_vec])

In [81]:
[(words[int(ind)], dist) for ind, dist in zip(list(indices[0]), list(distances[0]))]

[('doctor', 0.1344949),
 ('physician', 0.22574711),
 ('he', 0.26930726),
 ('his', 0.32090294),
 ('admitted', 0.3226241),
 ('who', 0.32832408),
 ('medical', 0.32892072),
 ('him', 0.3305949),
 ('surgeon', 0.33283055),
 ('an', 0.33643085)]

## Bias

Again, just looking at individual words is a **noisy** approach (I'm using it as a simple illustration).  [Researchers from Princeton and University of Bath](https://www.princeton.edu/~aylinc/papers/caliskan-islam_semantics.pdf) use **small baskets of terms** to represent concepts.  They first confirmed that flowers are more pleasant than insects, and musical instruments are more pleasant from weapons.

They then found that European American names are "more pleasant" than African American names, as captured by how close the word vectors are (as embedded by GloVe, which is a library from Stanford, along the same lines as Word2Vec).

    We show for the first time that if AI is to exploit via our language the vast 
    knowledge that culture has compiled, it will inevitably inherit human-like 
    prejudices. In other words, if AI learns enough about the properties of language 
    to be able to understand and produce it, it also acquires cultural associations 
    that can be offensive, objectionable, or harmful.

[Researchers from Boston University and Microsoft Research](https://arxiv.org/pdf/1606.06121.pdf) found the pairs most analogous to *He : She*.  They found gender bias, and also proposed a way to debias the vectors.

Rob Speer, CTO of Luminoso, tested for ethnic bias by finding correlations for a list of positive and negative words:

    The tests I implemented for ethnic bias are to take a list of words, such as 
    “white”, “black”, “Asian”, and “Hispanic”, and find which one has the strongest 
    correlation with each of a list of positive and negative words, such as “cheap”, 
    “criminal”, “elegant”, and “genius”. I did this again with a fine-grained version 
    that lists hundreds of words for ethnicities and nationalities, and thus is more 
    difficult to get a low score on, and again with what may be the trickiest test of 
    all, comparing words for different religions and spiritual beliefs.

**Ways to address bias**

There are a few different approaches:

- Debias word embeddings
  - [Technique in Bolukbasi, et al.](https://arxiv.org/abs/1606.06121)
  - [ConceptNet Numberbatch (Rob Speer)](https://blog.conceptnet.io/2017/04/24/conceptnet-numberbatch-17-04-better-less-stereotyped-word-vectors/)
- Argument that “awareness is better than blindness”: debiasing should happen at time of action, not at perception. ([Caliskan-Islam, Bryson, Narayanan](https://www.princeton.edu/~aylinc/papers/caliskan-islam_semantics.pdf))

Either way, you need to be on the lookout for bias and have a plan to address it!

If you are interested in the topic of bias in AI, I gave a workshop [you can watch here](https://www.youtube.com/watch?v=25nC0n9ERq4) that covers this material and goes into more depth about bias.

# Movie Reviews Sentiment Analysis Demo

This demo has been adapted (and simplified) from part of Lesson 5 of [Practical Deep Learning for Coders](http://course.fast.ai/index.html)

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. 

We will be using [Keras](https://keras.io/), a high-level neural network API. Two of the guiding principles of Keras are **user-friendliness** (it's designed for humans, not machines) and **works with Python**.  Yay for both of these!

Keras can run on top of many other neural network frameworks, including TensorFlow, Theano, R, MxNet, or CNTK.  I am using it on top of TensorFlow here.

Keras comes with some helpers for the IMDB dataset.

In [160]:
from keras.datasets import imdb
from keras.utils.data_utils import get_file
idx = imdb.get_word_index()

Using TensorFlow backend.


1646592/1641221 [==============================] - 1s 0us/step


In [161]:
import keras.backend as K

def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    cfg.gpu_options.per_process_gpu_memory_fraction = 0.6
    K.set_session(K.tf.Session(config=cfg))
    
limit_mem()

This is the word list:

In [162]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [163]:
idx2word = {v: k for k, v in idx.items()}

We download the reviews using code from https://keras.io/datasets/:

In [164]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

65560576/65552540 [==============================] - 31s 0us/step


In [165]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [166]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [167]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [168]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [169]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [170]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

Look at distribution of lengths of sentences.

In [171]:
trn[:10]

[array([4999,  309,    6,    3, 1069,  209,    9, 2175,   30,    1,  169,
          55,   14,   46,   82, 4999,   41,  393,  110,  138,   14, 4999,
          58, 4477,  150,    8,    1, 4999, 4999,  482,   69,    5,  261,
          12, 4999, 4999, 2003,    6,   73, 2436,    5,  632,   71,    6,
        4999,    1, 4999,    5, 2004, 4999,    1, 4999, 1534,   34,   67,
          64,  205,  140,   65, 1232, 4999, 4999,    1, 4999,    4,    1,
         223,  901,   29, 3024,   69,    4,    1, 4999,   10,  694,    2,
          65, 1534,   51,   10,  216,    1,  387,    8,   60,    3, 1472,
        3724,  802,    5, 3521,  177,    1,  393,   10, 1238, 4999,   30,
         309,    3,  353,  344, 2989,  143,  130,    5, 4999,   28,    4,
         126, 4999, 1472, 2375,    5, 4999,  309,   10,  532,   12,  108,
        1470,    4,   58,  556,  101,   12, 4999,  309,    6,  227, 4187,
          48,    3, 2237,   12,    9,  215]),
 array([4999,   39, 4999,   14,  739, 4999, 3428,   44,   74,   32

In [172]:
lens = np.array([len(review) for review in trn])

In [173]:
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [174]:
from keras.preprocessing import sequence

In [175]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [176]:
trn.shape

(25000, 500)

## Create a model

### Single conv layer with max pooling

*Convolutional neural networks* (abbreviated CNNs) are a powerful type of neural networks that do well with ordered data.  They have traditionally been used primarily for image data, but more recently are showing great results on natural language data.  [Facebook AI recently announced results](https://code.facebook.com/posts/1978007565818999/a-novel-approach-to-neural-machine-translation/) of using a CNN to speed up language translation 9x faster than the current state-of-the-art. 

We'll use a 1D CNN, since a sequence of words is 1D.

For this workshop, we will treat the CNN as a black box.  If you want to learn more about what is going on inside it, check out [Practical Deep Learning for Coders](http://course.fast.ai/) (the only pre-req is 1 year of coding experience).

In [177]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.optimizers import Adam

In [178]:
conv1 = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.4),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning:

The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning:

Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, activation="relu", padding="same")`



In [179]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [180]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Instructions for updating:
Use tf.cast instead.


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.



Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 38s 2ms/step - loss: 0.4394 - acc: 0.7734 - val_loss: 0.2615 - val_acc: 0.8890
Epoch 2/4
25000/25000 [==============================] - 38s 2ms/step - loss: 0.2258 - acc: 0.9159 - val_loss: 0.2655 - val_acc: 0.8878
Epoch 3/4
25000/25000 [==============================] - 40s 2ms/step - loss: 0.1674 - acc: 0.9397 - val_loss: 0.2838 - val_acc: 0.8880
Epoch 4/4
25000/25000 [==============================] - 50s 2ms/step - loss: 0.1194 - acc: 0.9582 - val_loss: 0.3549 - val_acc: 0.8736


In deep learning, often as you get closer to the answer, you need to reduce your *learning rate*, which is the step size for how the algorithm changes it's guess each time.  When you are far from the answer, you want to take large steps to get to the right vicinity.  Once you are close to the answer, you want to take small steps so you don't overshoot the answer.

In [181]:
conv1.optimizer.lr=1e-4

In [182]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
  128/25000 [..............................] - ETA: 32s - loss: 0.0518 - acc: 0.9922

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.



25000/25000 [==============================] - 36s 1ms/step - loss: 0.0792 - acc: 0.9731 - val_loss: 0.4003 - val_acc: 0.8763
Epoch 2/4
25000/25000 [==============================] - 37s 1ms/step - loss: 0.0494 - acc: 0.9836 - val_loss: 0.4600 - val_acc: 0.8797
Epoch 3/4
25000/25000 [==============================] - 37s 1ms/step - loss: 0.0308 - acc: 0.9894 - val_loss: 0.5515 - val_acc: 0.8776
Epoch 4/4
25000/25000 [==============================] - 38s 2ms/step - loss: 0.0259 - acc: 0.9914 - val_loss: 0.5757 - val_acc: 0.8791


The [Stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf)(2011) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 88.3%.  We have surpassed that!

Note that accuracy of 88.9% means an error rate of 11.1% (it's often more helpful to talk about error rates).

### Using our GloVe word embeddings

We could improve our model by using the GloVe word embeddings from above, since this capture semantic meaning, and have been trained for much longer on a much larger dataset than what we are using here.

We are going to use a version of GloVe where the embeddings have just 50 dimensions (as opposed to 100).  It's the same idea as before.

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [102]:
def create_emb():
    n_fact = vecs50.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs50[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = np.random.normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = np.random.normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [103]:
emb = create_emb()

NameError: name 'idx2word' is not defined

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [104]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.4, weights=[emb]),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')])

NameError: name 'Sequential' is not defined

In [105]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

NameError: name 'model' is not defined

In [106]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

NameError: name 'model' is not defined

We decrease the learning rate now that we are getting closer to the answer.

In [107]:
model.optimizer.lr=1e-4

NameError: name 'model' is not defined

In [108]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

NameError: name 'model' is not defined

Our error rate has improved from 11.1% to 10.3%, a 7% improvement 

(this value was fluctuating, but I typically got that it was between 4-10%)

In [109]:
(11.1 - 10.3)/11.1

0.07207207207207197